In [ ]:
!pip install pyspark py4j


In [ ]:
!apt-get -qq update
!apt-get -y install postgresql postgresql-contrib openjdk-8-jdk

!pip install -q pyspark findspark psycopg2-binary
!service postgresql start
!ps aux | grep postgres
!apt-get update
!apt-get install -y postgresql postgresql-contrib

In [22]:
from pyspark.sql import SparkSession


In [23]:
spark = SparkSession.builder \
    .appName("PostgresSpark") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.27") \
    .getOrCreate()


In [24]:
from pyspark.sql.functions import *

In [25]:
df = spark.read.json('/content/drive/MyDrive/spark/*.json').withColumn('file_name',input_file_name())

In [26]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- forks: long (nullable = true)
 |-- full_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- language: string (nullable = true)
 |-- open_issues: long (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- topics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- type: string (nullable = true)
 |-- username: string (nullable = true)
 |-- file_name: string (nullable = false)



In [27]:
programming_lang = (df.groupBy('language').agg(count(col('language')).alias('count_of_repos_using_it')).sort(desc('count_of_repos_using_it') ))

In [28]:
organizations_stars= df.where(col('type') == "Organization").groupBy(col('username')).agg(sum('stars').alias('count_of_stars')).sort(desc('count_of_stars'))

In [29]:
search_terms_relevance = df.groupBy('file_name').agg(sum((1.5 * col('forks') + 1.32 * col('subscribers') + 1.04 * col('stars'))).alias('relevance score')).sort(desc('count_of_stars'))

In [31]:
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'mohsen123';"
!sudo -u postgres psql -c "CREATE USER mohsen WITH PASSWORD 'mohsen123';"
!sudo -u postgres psql -c "ALTER USER mohsen CREATEDB;"
!sudo -u postgres psql -c "CREATE DATABASE spark_db OWNER mohsen;"

ALTER ROLE
CREATE ROLE
ALTER ROLE
CREATE DATABASE


In [41]:
!service postgresql start


 * Starting PostgreSQL 14 database server
   ...done.


In [44]:
!sudo -u postgres psql -c "CREATE USER mohsen_essam WITH PASSWORD 'mohsen123';"
!sudo -u postgres psql -c "ALTER USER mohsen_essam CREATEDB;"
!sudo -u postgres psql -c "CREATE DATABASE  OWNER mohsen_essam;"


CREATE ROLE
ALTER ROLE
ERROR:  database "spark_db" already exists


In [45]:
!sudo -u postgres psql -d spark_db -c "\dt"


Did not find any relations.


In [47]:
!sudo sed -i 's/md5/trust/g' /etc/postgresql/*/main/pg_hba.conf


In [48]:
!service postgresql restart


 * Restarting PostgreSQL 14 database server
   ...done.


In [50]:
!service postgresql status


14/main (port 5432): online


In [51]:
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'mypassword';"


ALTER ROLE


In [52]:
import psycopg2

conn = psycopg2.connect(
    dbname="spark_db",
    user="postgres",
    password="mypassword",
    host="localhost"
)

cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())

conn.close()


('PostgreSQL 14.17 (Ubuntu 14.17-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [54]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Postgres_Spark") \
    .config("spark.jars", "https://jdbc.postgresql.org/download/postgresql-42.2.20.jar") \
    .getOrCreate()


In [59]:
programming_lang.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table2") .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .save()


In [60]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table2") \
    .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.show()


+----------------+-----------------------+
|        language|count_of_repos_using_it|
+----------------+-----------------------+
|      JavaScript|                   5293|
|          Python|                   4861|
|      TypeScript|                   2816|
|              Go|                   1868|
|            Java|                   1418|
|Jupyter Notebook|                   1291|
|           Scala|                   1178|
|             C++|                    952|
|            Dart|                    855|
|          Kotlin|                    830|
|           Julia|                    806|
|            HTML|                    524|
|           Shell|                    520|
|              C#|                    336|
|               C|                    321|
|             CSS|                    227|
|     Objective-C|                    192|
|            Ruby|                    174|
|            Rust|                    165|
|             PHP|                    159|
+----------

In [61]:
organizations_stars.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table3") .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .save()

In [63]:
df2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table3") \
    .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df2.show()


+-----------+--------------+
|   username|count_of_stars|
+-----------+--------------+
|  microsoft|       1011553|
| tensorflow|        859765|
|   facebook|        627455|
|     apache|        433004|
|    angular|        352533|
|huggingface|        306537|
|     google|        289449|
|    pytorch|        255202|
|gothinkster|        218946|
|storybookjs|        208451|
| kubernetes|        206820|
| keras-team|        195019|
|     vercel|        180974|
|    alibaba|        172746|
|    Tencent|        159319|
|    flutter|        159010|
|     nodejs|        144071|
|     docker|        142690|
|    ohmyzsh|        141449|
|       dmlc|        139827|
+-----------+--------------+
only showing top 20 rows



In [64]:
search_terms_relevance.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table4") .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .save()

In [67]:
from os import truncate
df3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/spark_db") \
    .option("dbtable", "table4") \
    .option("user", "postgres") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df3.show(truncate= False)

+---------------------------------------------------------+------------------+
|file_name                                                |count_of_stars    |
+---------------------------------------------------------+------------------+
|file:///content/drive/MyDrive/spark/R.json               |1.187633824E7     |
|file:///content/drive/MyDrive/spark/Machine-Learning.json|6925617.799999998 |
|file:///content/drive/MyDrive/spark/Deep-Learning.json   |6559100.0200000005|
|file:///content/drive/MyDrive/spark/Golang.json          |5407407.259999992 |
|file:///content/drive/MyDrive/spark/Typescript.json      |4909697.760000003 |
|file:///content/drive/MyDrive/spark/Docker.json          |4403817.560000001 |
|file:///content/drive/MyDrive/spark/Cpp.json             |4377811.440000002 |
|file:///content/drive/MyDrive/spark/NodeJS.json          |4331014.679999995 |
|file:///content/drive/MyDrive/spark/Tensorflow.json      |3918987.260000002 |
|file:///content/drive/MyDrive/spark/Kubernetes.json

In [73]:
conn = psycopg2.connect(
    dbname="spark_db",
    user="postgres",
    password="mypassword",
    host="localhost"
)


In [82]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table4")

for i in cursor.fetchall():
  print(i)





('file:///content/drive/MyDrive/spark/R.json', 11876338.24)
('file:///content/drive/MyDrive/spark/Machine-Learning.json', 6925617.799999998)
('file:///content/drive/MyDrive/spark/Deep-Learning.json', 6559100.0200000005)
('file:///content/drive/MyDrive/spark/Golang.json', 5407407.259999992)
('file:///content/drive/MyDrive/spark/Typescript.json', 4909697.760000003)
('file:///content/drive/MyDrive/spark/Docker.json', 4403817.560000001)
('file:///content/drive/MyDrive/spark/Cpp.json', 4377811.440000002)
('file:///content/drive/MyDrive/spark/NodeJS.json', 4331014.679999995)
('file:///content/drive/MyDrive/spark/Tensorflow.json', 3918987.260000002)
('file:///content/drive/MyDrive/spark/Kubernetes.json', 3540183.0199999996)
('file:///content/drive/MyDrive/spark/PyTorch.json', 3357329.079999997)
('file:///content/drive/MyDrive/spark/React-Native.json', 3097924.4600000037)
('file:///content/drive/MyDrive/spark/Angular.json', 2888727.660000004)
('file:///content/drive/MyDrive/spark/Kotlin.json',